# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each scenario separately. These files will be used in the Jupyter Notebook.

3. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion

4. Rerun batch sims in EnergyPlus v9.5.
5. Add scenario names, elevations, and glazing systems to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used.

lighting_redux = .3

---

## Combine CSV Files in Directory Commercial Electric Heating

In [7]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ComElecElec/") if f.endswith(".csv")]

# files

In [8]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ComElecElec/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 105120  rows, 108 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,cle_com_elec_10,E,Exist_LowE,01/01 01:00:00,-1.2000,0.0,11869.19907,0.0,0.0,0.0,...,77.977719,-1.0,-1.0,-0.886806,-1.0,0.0,1.533456e+06,0,6.662803e+05,0.0
1,cle_com_elec_10,E,Exist_LowE,01/01 02:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.840990,-1.0,-1.0,-0.886806,-1.0,0.0,1.787081e+06,0,8.883738e+05,0.0
2,cle_com_elec_10,E,Exist_LowE,01/01 03:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.609970,-1.0,-1.0,-0.886806,-1.0,0.0,1.437250e+06,0,6.662803e+05,0.0
3,cle_com_elec_10,E,Exist_LowE,01/01 04:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.922504,-1.0,-1.0,-0.886806,-1.0,0.0,1.860285e+06,0,8.883738e+05,0.0
4,cle_com_elec_10,E,Exist_LowE,01/01 05:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.471419,-1.0,-1.0,-0.886806,-1.0,0.0,1.584762e+06,0,6.662803e+05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,cle_com_elec_09,N,Exist_LowE,12/31 20:00:00,3.4875,0.0,11869.19907,0.0,0.0,0.0,...,84.124350,-1.0,-1.0,-0.714479,-1.0,0.0,0.000000e+00,0,0.000000e+00,0.0
8756,cle_com_elec_09,N,Exist_LowE,12/31 21:00:00,3.5250,0.0,11869.19907,0.0,0.0,0.0,...,76.344404,-1.0,-1.0,-0.714479,-1.0,0.0,1.276364e+06,0,1.406848e+06,0.0
8757,cle_com_elec_09,N,Exist_LowE,12/31 22:00:00,3.6750,0.0,11869.19907,0.0,0.0,0.0,...,74.403663,-1.0,-1.0,-0.714479,-1.0,0.0,5.762689e+05,0,1.406848e+06,0.0
8758,cle_com_elec_09,N,Exist_LowE,12/31 23:00:00,4.2125,0.0,11869.19907,0.0,0.0,0.0,...,79.769622,-1.0,-1.0,-0.714479,-1.0,0.0,7.915104e+05,0,7.034239e+05,0.0


In [9]:
#list(all_ComElec.columns.values)

In [10]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [11]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

-----

## Get the Wanted Data (columns)

In [12]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [13]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [14]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [15]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [16]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

# get_annual_ComElec

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [17]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [18]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [19]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [20]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [21]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [22]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [23]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>

In [24]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [25]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [26]:
# Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

#allAnnualDataComElec

In [27]:
# list(allAnnualDataComElec.columns.values)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [28]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [29]:
# Rename columns to indicate baseline to prepare for merge\n"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={"Scenario":"Scenario",
 "Date_Time":"Date_Time",
 "Elevation":"Elevation",
 "GlazingSystem":"B_GlazingSystem",
 "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
 "HeatingElecEnergy(kBtu)":"B_HeatingElecEnergy(kBtu)",
 "HeatingGasEnergy(kBtu)":"B_HeatingGasEnergy(kBtu)",
 "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
 "LightingEnergy(kBtu)":"B_LightingEnergy(kBtu)",
 "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
 "FanEnergy(kBtu/sf)":"B_FanEnergy(kBtu/sf)",
 "HeatingElecEnergy(kBtu/sf)":"B_HeatingElecEnergy(kBtu/sf)",
 "HeatingGasEnergy(kBtu/sf)":"B_HeatingGasEnergy(kBtu/sf)",
 "CoolingEnergy(kBtu/sf)":"B_CoolingEnergy(kBtu/sf)",
 "LightingEnergy(kBtu/sf)":"B_LightingEnergy(kBtu/sf)",
 "EUI(kBtu/sf)":"B_EUI(kBtu/sf)",
 "CO2_Electric(lb/sf)":"B_CO2_Electric(lb/sf)",
 "FanCost($)":"B_FanCost($)",
 "HeatingElecCost($)":"B_HeatingElecCost($)",
 "HeatingGasCost($)":"B_HeatingGasCost($)",
 "CoolingCost($)":"B_CoolingCost($)",
 "LightingCost($)":"B_LightingCost($)",
 "AnnualCost($)":"B_AnnualCost($)",
 "FanCost($/sf)":"B_FanCost($/sf)",
 "HeatingElecCost($/sf)":"B_HeatingElecCost($/sf)",
 "HeatingGasCost($/sf)":"B_HeatingGasCost($/sf)",
 "CoolingCost($/sf)":"B_CoolingCost($/sf)",
 "LightingCost($/sf)":"B_LightingCost($/sf)",
 "AnnualCost($/sf)":"B_AnnualCost($/sf)"
                            })

#allBaselineAnnualDataComElec

In [30]:
# Merge the dataframes to include the baseline data.
AnnualWithBaselineCom = pd.merge(allAnnualDataComElec, allBaselineAnnualDataComElec, on=["Date_Time", "Elevation", "Scenario"], how="inner")

# AnnualWithBaselineCom

In [31]:
# list(AnnualWithBaselineCom.columns.values)

In [32]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaselineCom["deltaEnergy(kBtu)"] = AnnualWithBaselineCom["AnnualEnergy(kBtu)"] - AnnualWithBaselineCom["B_AnnualEnergy(kBtu)"]
AnnualWithBaselineCom["deltaEUI(kBtu/sf)"] = AnnualWithBaselineCom["EUI(kBtu/sf)"] - AnnualWithBaselineCom["B_EUI(kBtu/sf)"]
AnnualWithBaselineCom["deltaCO2Elec(lb/sf)"] = AnnualWithBaselineCom["CO2_Electric(lb/sf)"] - AnnualWithBaselineCom["B_CO2_Electric(lb/sf)"]
AnnualWithBaselineCom["deltaCost($)"] = AnnualWithBaselineCom["AnnualCost($)"] - AnnualWithBaselineCom["B_AnnualCost($)"]
AnnualWithBaselineCom["deltaCost($/sf)"] = AnnualWithBaselineCom["AnnualCost($/sf)"] - AnnualWithBaselineCom["B_AnnualCost($/sf)"]

# AnnualWithBaselineCom

In [33]:
# Convert NANs to 0
AnnualWithBaselineCom =AnnualWithBaselineCom.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario
Export to file

In [34]:
# Group by scenario
annualDataCom = AnnualWithBaselineCom.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

#annualData.sum().round(2)

In [35]:
annualDataComFinal = annualDataCom.sum().round(2)

#annualDataComFinal

In [36]:
# Export grouped file to csv
annualDataComFinal.to_csv("Scraper_Output/cle_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [37]:
# Start with renaming df
get_peak_ComElec = get_annual_ComElec
#get_peak_ComElec

In [38]:
# Add up heat, cool and fans for a total column
get_peak_ComElec["TotalEnergy(kBtu)"] = (get_peak_ComElec["FanEnergy(kBtu)"] +
                                    get_peak_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_peak_ComElec["LightingEnergy(kBtu)"] +
                                    get_peak_ComElec["CoolingEnergy(kBtu)"])

#get_peak_ComElec

In [39]:
# Get scenario and find the max
scenario1 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_01"]
scenario2 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_02"]
scenario3 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_03"]
scenario4 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_04"]
scenario5 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_05"]
scenario6 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_06"]
scenario7 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_07"]
scenario8 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_08"]
scenario9 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_09"]
scenario10 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_10"]
scenario11 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_11"]
scenario12 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_12"]

In [40]:
scenarioPeak1 = (scenario1[scenario1["TotalEnergy(kBtu)"] == scenario1["TotalEnergy(kBtu)"].max()])
scenarioPeak2 = (scenario2[scenario2["TotalEnergy(kBtu)"] == scenario2["TotalEnergy(kBtu)"].max()])
scenarioPeak3 = (scenario3[scenario3["TotalEnergy(kBtu)"] == scenario3["TotalEnergy(kBtu)"].max()])
scenarioPeak4 = (scenario4[scenario4["TotalEnergy(kBtu)"] == scenario4["TotalEnergy(kBtu)"].max()])
scenarioPeak5 = (scenario5[scenario5["TotalEnergy(kBtu)"] == scenario5["TotalEnergy(kBtu)"].max()])
scenarioPeak6 = (scenario6[scenario6["TotalEnergy(kBtu)"] == scenario6["TotalEnergy(kBtu)"].max()])
scenarioPeak7 = (scenario7[scenario7["TotalEnergy(kBtu)"] == scenario7["TotalEnergy(kBtu)"].max()])
scenarioPeak8 = (scenario8[scenario8["TotalEnergy(kBtu)"] == scenario8["TotalEnergy(kBtu)"].max()])
scenarioPeak9 = (scenario9[scenario9["TotalEnergy(kBtu)"] == scenario9["TotalEnergy(kBtu)"].max()])
scenarioPeak10 = (scenario10[scenario10["TotalEnergy(kBtu)"] == scenario10["TotalEnergy(kBtu)"].max()])
scenarioPeak11 = (scenario11[scenario11["TotalEnergy(kBtu)"] == scenario11["TotalEnergy(kBtu)"].max()])
scenarioPeak12 = (scenario12[scenario12["TotalEnergy(kBtu)"] == scenario12["TotalEnergy(kBtu)"].max()])

In [41]:
PeakDemandElec =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),TotalEnergy(kBtu)
27151,cle_com_elec_01,02/06 08:00:00,N,Exist,1.856860,19.611957,0.0,0.0,0.02025,21.489066,0.005635,0.059511,0.0,0.0,0.000061,0.065207,0.027454,21.489066
35911,cle_com_elec_02,02/06 08:00:00,E,Exist,1.074452,20.151331,0.0,0.0,0.02025,21.246033,0.003260,0.061148,0.0,0.0,0.000061,0.064470,0.027144,21.246033
44671,cle_com_elec_03,02/06 08:00:00,S,Exist,1.739669,19.149761,0.0,0.0,0.02025,20.909680,0.005279,0.058109,0.0,0.0,0.000061,0.063449,0.026714,20.909680
53431,cle_com_elec_04,02/06 08:00:00,W,Exist,1.558531,19.621173,0.0,0.0,0.02025,21.199953,0.004729,0.059539,0.0,0.0,0.000061,0.064330,0.027085,21.199953
62191,cle_com_elec_05,02/06 08:00:00,N,Exist_Clear,1.680864,15.162749,0.0,0.0,0.02025,16.863863,0.005100,0.046010,0.0,0.0,0.000061,0.051172,0.021545,16.863863
70951,cle_com_elec_06,02/06 08:00:00,E,Exist_Clear,1.047936,15.745612,0.0,0.0,0.02025,16.813797,0.003180,0.047779,0.0,0.0,0.000061,0.051020,0.021481,16.813797
79711,cle_com_elec_07,02/06 08:00:00,S,Exist_Clear,1.577612,14.383507,0.0,0.0,0.02025,15.981368,0.004787,0.043646,0.0,0.0,0.000061,0.048495,0.020418,15.981368
88471,cle_com_elec_08,02/06 08:00:00,W,Exist_Clear,1.388985,15.378881,0.0,0.0,0.02025,16.788115,0.004215,0.046666,0.0,0.0,0.000061,0.050943,0.021448,16.788115
97231,cle_com_elec_09,02/06 08:00:00,N,Exist_LowE,1.333435,15.757028,0.0,0.0,0.02025,17.110712,0.004046,0.047814,0.0,0.0,0.000061,0.051921,0.021860,17.110712
871,cle_com_elec_10,02/06 08:00:00,E,Exist_LowE,0.842016,16.173465,0.0,0.0,0.02025,17.035730,0.002555,0.049077,0.0,0.0,0.000061,0.051694,0.021765,17.035730


In [42]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/cle_peak_com.csv", header=True, index=True)